# Compare ModelLevel files for AeroCom analysis

This notebook illustrates common issues that can occur when reading 4D gridded model data into pyaerocom (in the following referred to as *ModelLevel*), i.e. data that contains a vertical dimension in addition to latitude, longitude and time dimension. 2 example files are read using pyaerocom that contain 4D ModelLevel fields (from recent AeroCom experiments), in order to illustrate common issues that occur with the vertical dimension, if the files do not strictly follow the [CF conventions](http://cfconventions.org/index.html).

## Preface on CF conventions and data preparation

pyaerocom uses the [iris](https://scitools.org.uk/iris/docs/latest/index.html) for reading of NetCDF files. iris (and pyaerocom) requires, that the NetCDF files follow the CF conventions. 

**Most importantly, for a successful import of the data, it is important that all variables, attributes, etc in the file follow the CF conventions and include relevant definitions of `standard_name`, `units`, etc. wherever applicable.** 

You can check here for existing standards:

http://cfconventions.org/Data/cf-standard-names/65/build/cf-standard-name-table.html

As mentioned above, here we focus on supported standard ***vertical coordinates***, which allow to convert model levels to pressure fields, and ultimately, to altitudes above sea level (for each grid point). To read more about CF compliant vertical coordinates, and how they should be defined in the NetCDF files, see here:

http://cfconventions.org/Data/cf-conventions/cf-conventions-1.0/build/apd.html

If you intend to submit data to AeroCom experiments, please make use of the online CF checker, which can be found here:

http://pumatest.nerc.ac.uk/cgi-bin/cf-checker.pl?cfversion=auto

## Data used in this notebook

In this notebook, 4D Aerosol extinction fields from the following two models are read (the corresponding files are specified below and can be found in this repository):

- GISS-MATRIX_GLOFIR0p5  (working)
- OsloCTM3v1.01-met2010_AP3-CTRL (partly working)

###  Data files (all available in this repo)

In [26]:
FILE1_GISS_OK = 'aerocom3_GISS-MATRIX_GLOFIR0p5_ec550aer_ModelLevel_2008_daily.nc'
FILE2_OSLOCTM_PARTLYOK = 'aerocom3_OsloCTM3v1.01-met2010_AP3-CTRL_ec550aer_ModelLevel_2010_monthly_CORR.nc'

**NOTE**: the 2. file from OsloCTM (`FILE2_OSLOCTM_PARTLYOK`) was created from file 
```
aerocom3_OsloCTM3v1.01-met2010_AP3-CTRL_ec550aer_ModelLevel_2010_monthly.nc
```
in script 
```
test_read.py
``` 

which both can be found in this repo.

In [2]:
import pyaerocom as pya

Initating pyaerocom configuration
Checking database access...
Checking access to: /lustre/storeA
Access to lustre database: True
Init data paths for lustre
Expired time: 0.018 s


### Example 1: GISS (EVERYTHING WELL DEFINED)

In [3]:
data_giss = pya.GriddedData(FILE1_GISS_OK, var_name='ec550aer')

/home/jonasg/anaconda3/lib/python3.6/site-packages/iris/fileformats/cf.py:1069: UserWarning: Ignoring formula terms variable 'ps' referenced by data variable 'b_bnds' via variable 'lev': Dimensions ('time', 'lat', 'lon') do not span ('lev', 'bnds')
  warnings.warn(msg)
/home/jonasg/anaconda3/lib/python3.6/site-packages/iris/fileformats/cf.py:1069: UserWarning: Ignoring formula terms variable 'ps' referenced by data variable 'a_bnds' via variable 'lev': Dimensions ('time', 'lat', 'lon') do not span ('lev', 'bnds')
  warnings.warn(msg)
/home/jonasg/anaconda3/lib/python3.6/site-packages/iris/fileformats/netcdf.py:599: UserWarning: Unable to find coordinate for variable 'ps'
  '{!r}'.format(name))
/home/jonasg/anaconda3/lib/python3.6/site-packages/iris/fileformats/netcdf.py:599: UserWarning: Unable to find coordinate for variable 'ps'
  '{!r}'.format(name))


In [4]:
print(data_giss)

pyaerocom.GriddedData: n/d
Grid data: Aerosol Extinction @550nm                                (time: 2; atmosphere_hybrid_sigma_pressure_coordinate: 40; latitude: 90; longitude: 144)
     Dimension coordinates:
          time                                                x                                               -             -              -
          atmosphere_hybrid_sigma_pressure_coordinate         -                                               x             -              -
          latitude                                            -                                               -             x              -
          longitude                                           -                                               -             -              x
     Auxiliary coordinates:
          surface_air_pressure                                x                                               -             x              x
          hybrid sigma coordinate A coefficient for lay

#### IMPORTANT REMARK

If you look in the output you can see and entry `air_pressure` under `Derived coordinates`. This is done automatiExamplecally by [iris](https://scitools.org.uk/iris/docs/latest/) library but only works if all required specificat
ions are available and follow the CF conventions. 

### Example 2: OsloCTM3 (PARTLY WORKING)

In [6]:
data_osloctm_corr = pya.GriddedData(FILE2_OSLOCTM_PARTLYOK, var_name='ec550aer')

In [7]:
print(data_osloctm_corr)

pyaerocom.GriddedData: n/d
Grid data: ec550aer                                             (time: 12; atmosphere_hybrid_sigma_pressure_coordinate: 60; latitude: 80; longitude: 160)
     Dimension coordinates:
          time                                            x                                                -             -              -
          atmosphere_hybrid_sigma_pressure_coordinate     -                                                x             -              -
          latitude                                        -                                                -             x              -
          longitude                                       -                                                -             -              x
     Attributes:
          Modelinfo1: OsloCTM3 is a 3D Chemical Transport Model
          Modelinfo2: It is driven by ECMWF meteorological data
          contactinfo: gunnar.myhre@cicero.oslo.no
          creation_date: Thu May  2 12:20:4

As you can see, the data contains the four coordinates, but here the pressure was not automatically derived. At the moment it is unclear why iris does not derive the 4D pressure field here (like it does for the previously shown GISS data), since the NetCDF file actually contains all relevant information to convert, which you can check when opening the file with xarray:

In [16]:
import xarray as xarr

ds_osloctm = xarr.open_dataset(FILE2_OSLOCTM_PARTLYOK)
print(ds_osloctm)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 80, lev: 60, lon: 160, time: 12)
Coordinates:
  * lon        (lon) float32 0.0 2.25 4.5 6.75 9.0 ... 351.0 353.25 355.5 357.75
  * lat        (lat) float32 -88.875 -86.625 -84.375 ... 84.375 86.625 88.875
  * lev        (lev) float32 0.0 1.0 2.0 3.0 4.0 ... 55.0 56.0 57.0 58.0 59.0
  * time       (time) datetime64[ns] 2009-12-31 2010-01-01 ... 2010-01-11
Dimensions without coordinates: bnds
Data variables:
    ap         (lev) float32 ...
    b          (lev) float32 ...
    lon_bnds   (lon, bnds) float32 ...
    lat_bnds   (lat, bnds) float32 ...
    ap_bnds    (lev, bnds) float32 ...
    b_bnds     (lev, bnds) float32 ...
    time_bnds  (time, bnds) datetime64[ns] ...
    ps         (time, lat, lon) float32 ...
    ec550aer   (time, lev, lat, lon) float32 ...
Attributes:
    title:          AEROCOM phase III output from OsloCTM3
    institution:    CICERO
    Modelinfo1:     OsloCTM3 is a 3D Chemical Transport Model
    Modelinfo2:     I

That is, the file contains the  variables `ap, b, ps` that are required to convert the model levels into a pressure field as defined in [here](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.0/build/apd.html) (Example D.3), i.e.:

```
p(n,k,j,i) = ap(k) + b(k)*ps(n,j,i)
```

The reason, why iris is not converting to pressure, may be because the formula term is wrong in OsloCTM:

In [21]:
ds_osloctm.lev

<xarray.DataArray 'lev' (lev: 60)>
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
       28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55.,
       56., 57., 58., 59.], dtype=float32)
Coordinates:
  * lev      (lev) float32 0.0 1.0 2.0 3.0 4.0 5.0 ... 55.0 56.0 57.0 58.0 59.0
Attributes:
    units:          1
    axis:           Z
    long_name:      hybrid sigma pressure coordinate
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    formula:        lev=ap+b*ps

I.e. comparing what is written in GISS:

In [24]:
ds_giss = xarr.open_dataset(FILE1_GISS_OK)
ds_giss.lev

<xarray.DataArray 'lev' (lev: 40)>
array([0.95, 0.93, 0.91, 0.89, 0.87, 0.85, 0.83, 0.81, 0.79, 0.77, 0.75, 0.73,
       0.71, 0.69, 0.67, 0.65, 0.63, 0.61, 0.59, 0.57, 0.55, 0.53, 0.51, 0.49,
       0.47, 0.45, 0.43, 0.41, 0.39, 0.37, 0.35, 0.33, 0.31, 0.29, 0.27, 0.25,
       0.23, 0.21, 0.19, 0.17])
Coordinates:
  * lev      (lev) float64 0.95 0.93 0.91 0.89 0.87 ... 0.25 0.23 0.21 0.19 0.17
Attributes:
    bounds:         lev_bnds
    units:          1
    axis:           Z
    positive:       down
    long_name:      hybrid sigma pressure coordinate
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    formula:        p(n,k,j,i) = a(k)*p0 + b(k)*ps(n,j,i)
    formula_terms:  p0: p0 a: a b: b ps: ps

2 things are obvious when comparing the definition of the `lev` coordinate between the two models:

   1. GISS follows the standard nomenclature for `formula` (cf. [here](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.0/build/apd.html)) and OsloCTM does not.
   2. OsloCTM misses mapping specs `formula_terms` that are required to link variable names in the file with the variables in the formula term (it uses the variable names directly).
   
**NOTE**: it is unclear at the moment whether this causes the problems.